In [2]:
import warnings
warnings.filterwarnings("ignore")

# VGG-1

In [6]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image as tf_image
from tensorflow import expand_dims as tf_expand_dims
from tensorflow import summary
from keras import backend as K
from keras import callbacks
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import os
import io
from datetime import datetime

logs_folder = "logs_vgg1"
if os.path.exists(logs_folder):
    os.system(f"rm -rf {logs_folder}")


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf_image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf_expand_dims(image, 0)
    return image


def image_grid(images, label_arr, pred_arr):
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(50, 50))
    for i in range(30):
        # Start next subplot.
        plt.subplot(
            8, 5, i + 1, title=f'Actual :+{label_arr[i]}+| Pred : {pred_arr[i]}')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)

    return figure

# define cnn model


def vgg_one():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# plot diagnostic learning curves


def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model


def run_test_harness():

    logdir = "logs_vgg1/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)
    model = vgg_one()
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_it = datagen.flow_from_directory('dataset_harp_vs_violin/train/',
                                           class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory('dataset_harp_vs_violin/test/',
                                          class_mode='binary', batch_size=64, target_size=(200, 200))
    start = time.time()
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                                  validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1, callbacks=[tensorboard_callback],)
    end = time.time()

    print('No. of Parameters: ', model.count_params())

    print("Time: ", end - start)

    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
    print('Test : > %.3f' % (acc * 100.0))

    _, acc = model.evaluate_generator(train_it, steps=len(train_it), verbose=1)
    print('Train : > %.3f' % (acc * 100.0))

    summarize_diagnostics(history)

    label_dict = {}
    for key, value in test_it.class_indices.items():
        label_dict[value] = key

    x, y = test_it.next()
    images = x
    labels = y
    predictions = model.predict(images)
    # print(x.shape,y.shape)
    label_vals = []
    pred_vals = []

    for j in range(len(labels)):
        image = images[j]
        label = labels[j]
        pred = round(predictions[j][0])
        # print(k, label_dict[int(label)], label_dict[pred])
        label_vals.append(label_dict[int(label)])
        pred_vals.append(label_dict[pred])
        # k += 1

    figure = image_grid(images, label_vals, pred_vals)
    file_writer = summary.create_file_writer(logdir)
    with file_writer.as_default():
        summary.image("Training data", plot_to_image(figure), step=0)


# entry point, run the test harness
run_test_harness()

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/50


2023-04-19 18:19:02.437201: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - ETA: 0s - loss: 1.0209 - accuracy: 0.5143

2023-04-19 18:19:09.640213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 9s 3s/step - loss: 1.0209 - accuracy: 0.5143 - val_loss: 0.7386 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 6s 3s/step - loss: 0.9466 - accuracy: 0.4714 - val_loss: 0.6714 - val_accuracy: 0.5333
Epoch 3/50
3/3 [==============================] - 6s 2s/step - loss: 0.7636 - accuracy: 0.5786 - val_loss: 0.6446 - val_accuracy: 0.7167
Epoch 4/50
3/3 [==============================] - 6s 3s/step - loss: 0.7468 - accuracy: 0.5571 - val_loss: 0.9556 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 6s 3s/step - loss: 0.6259 - accuracy: 0.6214 - val_loss: 0.6891 - val_accuracy: 0.5167
Epoch 6/50
3/3 [==============================] - 6s 3s/step - loss: 0.6664 - accuracy: 0.6357 - val_loss: 0.5766 - val_accuracy: 0.6167
Epoch 7/50
3/3 [==============================] - 6s 2s/step - loss: 0.5277 - accuracy: 0.7000 - val_loss: 0.5524 - val_accuracy: 0.7833
Epoch 8/50
3/3 [==============================] - 6s

2023-04-19 18:24:20.882037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 1s 1s/step - loss: 0.5110 - accuracy: 0.8000
Test : > 80.000


2023-04-19 18:24:23.821655: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 3s 834ms/step - loss: 0.0193 - accuracy: 1.0000
Train : > 100.000
2/2 [==============================] - 0s 95ms/step


# VGG-3

In [7]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image as tf_image
from tensorflow import expand_dims as tf_expand_dims
from tensorflow import summary
from keras import backend as K
from keras import callbacks
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import os
import io
from datetime import datetime

logs_folder = "logs_vgg3"
if os.path.exists(logs_folder):
    os.system(f"rm -rf {logs_folder}")


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf_image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf_expand_dims(image, 0)
    return image


def image_grid(images, label_arr, pred_arr):
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(50, 50))
    for i in range(30):
        # Start next subplot.
        plt.subplot(
            8, 5, i + 1, title=f'Actual :+{label_arr[i]}+| Pred : {pred_arr[i]}')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)

    return figure

# define cnn model


def vgg_three():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# plot diagnostic learning curves


def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model


def run_test_harness():

    logdir = "logs_vgg3/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)
    model = vgg_three()
    datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_it = datagen.flow_from_directory('dataset_harp_vs_violin/train/',
                                           class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = datagen.flow_from_directory('dataset_harp_vs_violin/test/',
                                          class_mode='binary', batch_size=64, target_size=(200, 200))
    start = time.time()
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                                  validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1, callbacks=[tensorboard_callback],)
    end = time.time()

    print('No. of Parameters: ', model.count_params())

    print("Time: ", end - start)

    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
    print('Test : > %.3f' % (acc * 100.0))

    _, acc = model.evaluate_generator(train_it, steps=len(train_it), verbose=1)
    print('Train : > %.3f' % (acc * 100.0))

    summarize_diagnostics(history)

    label_dict = {}
    for key, value in test_it.class_indices.items():
        label_dict[value] = key

    x, y = test_it.next()
    images = x
    labels = y
    predictions = model.predict(images)
    # print(x.shape,y.shape)
    label_vals = []
    pred_vals = []

    for j in range(len(labels)):
        image = images[j]
        label = labels[j]
        pred = round(predictions[j][0])
        # print(k, label_dict[int(label)], label_dict[pred])
        label_vals.append(label_dict[int(label)])
        pred_vals.append(label_dict[pred])
        # k += 1

    figure = image_grid(images, label_vals, pred_vals)
    file_writer = summary.create_file_writer(logdir)
    with file_writer.as_default():
        summary.image("Training data", plot_to_image(figure), step=0)


# entry point, run the test harness
run_test_harness()

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/50


2023-04-19 18:24:43.982604: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - ETA: 0s - loss: 0.6777 - accuracy: 0.5929

2023-04-19 18:24:51.501024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 9s 4s/step - loss: 0.6777 - accuracy: 0.5929 - val_loss: 2.0743 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 8s 2s/step - loss: 3.7703 - accuracy: 0.4714 - val_loss: 0.9525 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 8s 2s/step - loss: 1.1734 - accuracy: 0.5000 - val_loss: 1.0959 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 8s 2s/step - loss: 0.8645 - accuracy: 0.5357 - val_loss: 0.8128 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 7s 2s/step - loss: 0.7957 - accuracy: 0.5000 - val_loss: 0.7580 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - 7s 3s/step - loss: 0.7024 - accuracy: 0.5000 - val_loss: 0.7000 - val_accuracy: 0.4667
Epoch 7/50
3/3 [==============================] - 7s 3s/step - loss: 0.6837 - accuracy: 0.5786 - val_loss: 0.6927 - val_accuracy: 0.5000
Epoch 8/50
3/3 [==============================] - 7s

2023-04-19 18:31:47.340465: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 2s 2s/step - loss: 0.5231 - accuracy: 0.8000
Test : > 80.000


2023-04-19 18:31:50.626338: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 4s 964ms/step - loss: 0.0729 - accuracy: 0.9929
Train : > 99.286
2/2 [==============================] - 0s 181ms/step


# VGG 3 data aug

In [8]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image as tf_image
from tensorflow import expand_dims as tf_expand_dims
from tensorflow import summary
from keras import backend as K
from keras import callbacks
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import os
import io
from datetime import datetime

logs_folder = "logs_vgg3_aug"
if os.path.exists(logs_folder):
    os.system(f"rm -rf {logs_folder}")


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf_image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf_expand_dims(image, 0)
    return image


def image_grid(images, label_arr, pred_arr):
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(50, 50))
    for i in range(30):
        # Start next subplot.
        plt.subplot(
            8, 5, i + 1, title=f'Actual :+{label_arr[i]}+| Pred : {pred_arr[i]}')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)

    return figure

# define cnn model


def vgg_three_aug():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same', input_shape=(200, 200, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu',
              kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# plot diagnostic learning curves


def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

# run the test harness for evaluating a model


def run_test_harness():
    logdir = "logs_vgg3_aug/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)
    model = vgg_three_aug()
    train_datagen = ImageDataGenerator(rescale=1.0/255.0,
                                       width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
    test_datagen = ImageDataGenerator(rescale=1.0/255.0)
    train_it = train_datagen.flow_from_directory('dataset_harp_vs_violin/train/',
                                                 class_mode='binary', batch_size=64, target_size=(200, 200))
    test_it = test_datagen.flow_from_directory('dataset_harp_vs_violin/test/',
                                               class_mode='binary', batch_size=64, target_size=(200, 200))
    start = time.time()
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                                  validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1, callbacks=[tensorboard_callback],)
    end = time.time()

    print('No. of Parameters: ', model.count_params())

    print("Time: ", end - start)

    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=1)
    print('Test : > %.3f' % (acc * 100.0))

    _, acc = model.evaluate_generator(train_it, steps=len(train_it), verbose=1)
    print('Train : > %.3f' % (acc * 100.0))

    summarize_diagnostics(history)

    label_dict = {}
    for key, value in test_it.class_indices.items():
        label_dict[value] = key

    x, y = test_it.next()
    images = x
    labels = y
    predictions = model.predict(images)
    # print(x.shape,y.shape)
    label_vals = []
    pred_vals = []

    for j in range(len(labels)):
        image = images[j]
        label = labels[j]
        pred = round(predictions[j][0])
        # print(k, label_dict[int(label)], label_dict[pred])
        label_vals.append(label_dict[int(label)])
        pred_vals.append(label_dict[pred])
        # k += 1

    figure = image_grid(images, label_vals, pred_vals)
    file_writer = summary.create_file_writer(logdir)
    with file_writer.as_default():
        summary.image("Training data", plot_to_image(figure), step=0)


# entry point, run the test harness
run_test_harness()

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/50


2023-04-19 18:32:04.530664: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - ETA: 0s - loss: 0.9782 - accuracy: 0.5143

2023-04-19 18:32:14.020590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 11s 3s/step - loss: 0.9782 - accuracy: 0.5143 - val_loss: 0.7803 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 9s 3s/step - loss: 0.7471 - accuracy: 0.5000 - val_loss: 0.6988 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 9s 3s/step - loss: 0.7068 - accuracy: 0.4857 - val_loss: 0.6971 - val_accuracy: 0.4833
Epoch 4/50
3/3 [==============================] - 9s 2s/step - loss: 0.6817 - accuracy: 0.5143 - val_loss: 0.6841 - val_accuracy: 0.4833
Epoch 5/50
3/3 [==============================] - 9s 3s/step - loss: 0.6880 - accuracy: 0.5071 - val_loss: 0.6709 - val_accuracy: 0.7000
Epoch 6/50
3/3 [==============================] - 9s 3s/step - loss: 0.6795 - accuracy: 0.5643 - val_loss: 0.6946 - val_accuracy: 0.4833
Epoch 7/50
3/3 [==============================] - 9s 3s/step - loss: 0.6807 - accuracy: 0.5857 - val_loss: 0.6526 - val_accuracy: 0.6667
Epoch 8/50
3/3 [==============================] - 9

2023-04-19 18:39:37.081629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 1s 1s/step - loss: 0.5164 - accuracy: 0.7833
Test : > 78.333


2023-04-19 18:39:39.972335: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 5s 1s/step - loss: 0.3138 - accuracy: 0.8500
Train : > 85.000
2/2 [==============================] - 0s 152ms/step


# VGG-16 Transfer Model

In [10]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image as tf_image
from tensorflow import expand_dims as tf_expand_dims
from tensorflow import summary
from keras import backend as K
from keras import callbacks
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import os
import io
from datetime import datetime

logs_folder = "logs_vgg16"
if os.path.exists(logs_folder):
    os.system(f"rm -rf {logs_folder}")


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf_image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf_expand_dims(image, 0)
    return image


def image_grid(images, label_arr, pred_arr):
    """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
    # Create a figure to contain the plot.
    figure = plt.figure(figsize=(50, 50))
    for i in range(30):
        # Start next subplot.
        plt.subplot(
            8, 5, i + 1, title=f'Actual :+{label_arr[i]}+| Pred : {pred_arr[i]}')
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)

    return figure

# define cnn model


def define_model():
    # load model
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu',
                   kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# plot diagnostic learning curves


def summarize_diagnostics(history):
    # plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()


def run_test_harness():

    logdir = "logs_vgg16/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)
    model = define_model()
    datagen = ImageDataGenerator(featurewise_center=True)
    datagen.mean = [123.68, 116.779, 103.939]
    train_it = datagen.flow_from_directory('dataset_harp_vs_violin/train/',
                                           class_mode='binary', batch_size=64, target_size=(224, 224))

    test_it = datagen.flow_from_directory('dataset_harp_vs_violin/test/',
                                          class_mode='binary', batch_size=64, target_size=(224, 224))

    start = time.time()

    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                                  validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1, callbacks=[tensorboard_callback],)

    end = time.time()

    print('No. of Parameters: ', model.count_params())

    print("Time: ", end - start)

    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('Test : > %.3f' % (acc * 100.0))

    _, acc = model.evaluate_generator(train_it, steps=len(train_it), verbose=0)
    print('Train : > %.3f' % (acc * 100.0))

    summarize_diagnostics(history)

    label_dict = {}
    for key, value in test_it.class_indices.items():
        label_dict[value] = key

    x, y = test_it.next()
    images = x
    labels = y
    predictions = model.predict(images)
    # print(x.shape,y.shape)
    label_vals = []
    pred_vals = []

    for j in range(len(labels)):
        image = images[j]
        label = labels[j]
        pred = round(predictions[j][0])
        # print(k, label_dict[int(label)], label_dict[pred])
        label_vals.append(label_dict[int(label)])
        pred_vals.append(label_dict[pred])
        # k += 1

    figure = image_grid(images, label_vals, pred_vals)
    file_writer = summary.create_file_writer(logdir)
    with file_writer.as_default():
        summary.image("Training data", plot_to_image(figure), step=0)


# entry point, run the test harness
run_test_harness()

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/50


2023-04-19 18:43:27.932975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-04-19 18:43:30.393194: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2023-04-19 18:43:30.654859: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.


1/3 [=========>....................] - ETA: 14s - loss: 5.3098 - accuracy: 0.5625

2023-04-19 18:43:35.698610: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.
2023-04-19 18:43:35.961106: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 822083584 exceeds 10% of free system memory.


3/3 [==============================] - ETA: 0s - loss: 8.1770 - accuracy: 0.5929

...

2023-04-19 18:59:09.160696: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test : > 93.333


2023-04-19 18:59:15.568304: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Train : > 100.000
2/2 [==============================] - 5s 2s/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

# MLP Model

In [11]:
# baseline model with data augmentation for the dogs vs cats dataset
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import image as tf_image
from tensorflow import expand_dims as tf_expand_dims
from tensorflow import summary
from keras import backend as K
from keras import callbacks
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import time
import os
import io
from datetime import datetime

logs_folder = "logs_mlp"
if os.path.exists(logs_folder):
    os.system(f"rm -rf {logs_folder}")

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf_image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf_expand_dims(image, 0)
  return image
  
def image_grid(images,label_arr,pred_arr):
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(50,50))
  for i in range(30):
    # Start next subplot.
    plt.subplot(8, 5, i + 1, title=f'Actual :+{label_arr[i]}+| Pred : {pred_arr[i]}')
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)

  return figure

# define cnn model
def define_model():
  units = [512, 256, 128, 1]
  model = Sequential()
  model.add(Flatten(input_shape=(224, 224, 3)))
  for i in range(len(units)):
        model.add(Dense(units[i], activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  # Print the summary of the model to see the total number of parameters
  model.build()
  model.summary()
  opt = SGD(lr=0.001, momentum=0.9)
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  return model

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 

def run_test_harness():

    logdir = "logs_mlp/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = callbacks.TensorBoard(log_dir=logdir)
    model = define_model()
    datagen = ImageDataGenerator(featurewise_center=True)
    datagen.mean = [123.68, 116.779, 103.939]
    train_it = datagen.flow_from_directory('dataset_harp_vs_violin/train/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
    
    test_it = datagen.flow_from_directory('dataset_harp_vs_violin/test/',
		class_mode='binary', batch_size=64, target_size=(224, 224))
    
    start = time.time()

    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                validation_data=test_it, validation_steps=len(test_it), epochs=50, verbose=1, callbacks=[tensorboard_callback],)
    
    end = time.time()

    print('No. of Parameters: ', model.count_params())

    print("Time: ", end - start)

    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('Test : > %.3f' % (acc * 100.0))

    _, acc = model.evaluate_generator(train_it, steps=len(train_it), verbose=0)
    print('Train : > %.3f' % (acc * 100.0))

    summarize_diagnostics(history)

    label_dict = {}
    for key, value in test_it.class_indices.items():
        label_dict[value] = key

    x, y = test_it.next()
    images = x
    labels = y
    predictions=model.predict(images)
    # print(x.shape,y.shape)
    label_vals=[]
    pred_vals=[]

    for j in range(len(labels)):
        image = images[j]
        label = labels[j]
        pred=round(predictions[j][0])
        # print(k, label_dict[int(label)], label_dict[pred])
        label_vals.append(label_dict[int(label)])
        pred_vals.append(label_dict[pred])
        # k += 1

    figure = image_grid(images, label_vals, pred_vals)
    file_writer = summary.create_file_writer(logdir)
    with file_writer.as_default():
        summary.image("Training data", plot_to_image(figure), step=0)

# entry point, run the test harness
run_test_harness()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 150528)            0         
                                                                 
 dense_16 (Dense)            (None, 512)               77070848  
                                                                 
 dense_17 (Dense)            (None, 256)               131328    
                                                                 
 dense_18 (Dense)            (None, 128)               32896     
                                                                 
 dense_19 (Dense)            (None, 1)                 129       
                                                                 
 dense_20 (Dense)            (None, 1)                 2         
                                                                 
Total params: 77,235,203
Trainable params: 77,235,203


2023-04-19 18:59:38.623988: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - ETA: 0s - loss: 2.2685 - accuracy: 0.4714

2023-04-19 18:59:43.493123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


3/3 [==============================] - 6s 2s/step - loss: 2.2685 - accuracy: 0.4714 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 4s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 4s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 4s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/50
3/3 [==============================] - 4s 2s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/50
3/3 [==============================] - 4s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 7/50
3/3 [==============================] - 4s 1s/step - loss: 0.6932 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 8/50
3/3 [==============================] - 4s

2023-04-19 19:03:03.972398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Test : > 50.000


2023-04-19 19:03:05.858540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Train : > 50.000
2/2 [==============================] - 0s 49ms/step


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

# TABLE

|             Model            	| Training Time 	| Training Loss 	| Training Accuracy 	| Testing  Accuracy 	| Number of Model Parameter 	|
|:----------------------------:	|:-------------:	|:-------------:	|:-----------------:	|:-----------------:	|:-------------------------:	|
|         VGG(1-block)         	|     254.64    	|               	|        100        	|         75        	|          40961153         	|
|         VGG(3-block)         	|      366      	|               	|        100        	|         80        	|          10333505         	|
| VGG(3-block)  Data Augmented 	|      397      	|               	|       82.143      	|       71.667      	|          10333505         	|
|  Transfer Learning (VGG-16)  	|     181.25    	|               	|        100        	|        93.3       	|          17926209         	|
|           MLP Model          	|               	|               	|                   	|                   	|                           	|

# Q1

In General as the number of parameters increases the higher the capacity of learning complex neural networks. However, this may lead to overfitting as it would give good accuracy for training samples, but perform poorly for the test samples. We may note VGG16 model with transfer learning appears to have the highest test accuracy of all the models, which may be expected. Transfer learning is a powerful technique that involves leveraging a pre-trained model on a large dataset to improve the performance on a smaller, related dataset. 

The VGG-3 model with data augmentation appears to have a lower test accuracy than the VGG-3 model without data augmentation, which may be unexpected. Data augmentation is a common technique used to improve the generalization performance of deep learning models, but it may not always lead to better results, depending on the specifics of the dataset and the model architecture.

# Q2
